In [1]:
import logging

LOGGER = logging.getLogger('pynetdicom')
LOGGER.setLevel(logging.DEBUG)

# Verification SCU

In [2]:
from pynetdicom import AE
from pynetdicom.sop_class import VerificationSOPClass

# Initialise the Application Entity
ae = AE()

# Add a requested presentation context
ae.add_requested_context(VerificationSOPClass)

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate('127.0.0.1', 11112)

if assoc.is_established:
    # Use the C-ECHO service to send the request
    # returns the response status a pydicom Dataset
    status = assoc.send_c_echo()

    # Check the status of the verification request
    if status:
        # If the verification request succeeded this will be 0x0000
        print('C-ECHO request status: 0x{0:04x}'.format(status.Status))
    else:
        print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

E: Association request failed: unable to connect to remote
E: TCP Initialisation Error: Connection refused
E: Association Aborted


Association rejected, aborted or never connected


In [3]:
from pynetdicom import AE, VerificationPresentationContexts

ae = AE()
ae.requested_contexts = VerificationPresentationContexts

# Verification SCP

In [ ]:
from pynetdicom import AE
from pynetdicom.sop_class import VerificationSOPClass

# Initialise the Application Entity and specify the listen port
ae = AE()

# Add the supported presentation context
ae.add_supported_context(VerificationSOPClass)

# Start listening for incoming association requests in blocking mode
ae.start_server(('', 11112), block=True)

In [ ]:
import time

from pynetdicom import AE
from pynetdicom.sop_class import VerificationSOPClass

# Initialise the Application Entity and specify the listen port
ae = AE()

# Add the supported presentation context
ae.add_supported_context(VerificationSOPClass)

def on_c_echo(context, info):
    """Respond to a C-ECHO service request.

    Parameters
    ----------
    context : namedtuple
        The presentation context that the verification request was sent under.
    info : dict
        Information about the association and verification request.

    Returns
    -------
    status : int or pydicom.dataset.Dataset
        The status returned to the peer AE in the C-ECHO response. Must be
        a valid C-ECHO status value for the applicable Service Class as
        either an ``int`` or a ``Dataset`` object containing (at a
        minimum) a (0000,0900) *Status* element.
    """
    return 0x0000

ae.on_c_echo = on_c_echo

# Start listening for incoming association requests in non-blocking mode
scp = ae.start_server(('', 11112), block=False)

# Zzzzz
time.sleep(60)

# Shutdown the listen server
scp.shutdown()